In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


# !cp "/content/gdrive/My Drive/DataAugumentation.zip" .
#ADDED NEW SOURCE
!cp "/content/gdrive/My Drive/data_aug(3_slices_with_repeated)_acceleration_full_data_20000.zip" .
!unzip -qq DataAugumentation.zip
!unzip -qq data_aug_3_slices_with_repeated_cluster_5.zip
!rm DataAugumentation.zip
!rm data_aug_3_slices_with_repeated_cluster_5.zip
data_path = 'DataAugumentation'

Mounted at /content/gdrive
unzip:  cannot find or open DataAugumentation.zip, DataAugumentation.zip.zip or DataAugumentation.zip.ZIP.
unzip:  cannot find or open data_aug_3_slices_with_repeated_cluster_5.zip, data_aug_3_slices_with_repeated_cluster_5.zip.zip or data_aug_3_slices_with_repeated_cluster_5.zip.ZIP.
rm: cannot remove 'DataAugumentation.zip': No such file or directory
rm: cannot remove 'data_aug_3_slices_with_repeated_cluster_5.zip': No such file or directory


In [2]:
!unzip -qq data_aug_3_slices_with_repeated_acceleration_full_data_20000.zip

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
import joblib

SEQUENCE_LENGTH = 600
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 1e-4
PLOT_SAVE_DIR = 'predicted_vs_actual_plots'


def process_file(file_path):
    df = pd.read_csv(file_path)

    df['Time'] = df['Time'] - df['Time'].iloc[0]

    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)
    df['Acceleration'] = df['Speed'].diff().fillna(0)

    # features = df[['Engine speed', 'Throttle position', 'Accelerator pedal position', 'Speed']]
    features = df[['Engine speed', 'Speed', 'slope', 'Acceleration']]
    target = df['Momentary fuel consumption']

    features = features.iloc[:SEQUENCE_LENGTH]
    target = target.iloc[:SEQUENCE_LENGTH]
    # min_value = df[['Engine speed', 'Speed', 'slope', 'Acceleration','Momentary fuel consumption']].min()
    # max_value = df[['Engine speed', 'Speed', 'slope', 'Acceleration','Momentary fuel consumption']].max()
    # print(min_value,'minn\n\n')
    # print(max_value , 'maxx\n\n')
    return features.values, target.values



def pad_and_normalize(data, scaler, sequence_length=SEQUENCE_LENGTH):
    padded_data = pad_sequences(data, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')
    # normalized_data = scaler.transform(padded_data.reshape(-1, padded_data.shape[-1])).reshape(padded_data.shape)
    return normalized_data


scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_original = []
y_original = []
X_augmented = []
y_augmented = []

base_folder_path = '/content/'

# CHANGED TO 6 FOR NEW DATA
for i in range(6):
  if i == 5 :
    folder_path = os.path.join(base_folder_path, f'data_aug(3_slices_with_repeated)_cluster_{i}')
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            features, target = process_file(file_path)

            slices = filename.split('_')
            is_original_trip = slices[2] == slices[6] and slices[6] == slices[10]

            if is_original_trip:
                X_original.append(features)
                y_original.append(target)
            else:
                X_augmented.append(features)
                y_augmented.append(target)

# # Pad and convert lists to numpy arrays
# # X_original = pad_sequences(X_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
# # y_original = pad_sequences(y_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
# X_augmented = pad_sequences(X_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
# y_augmented = pad_sequences(y_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')

# num_test = int(0.05 * len(X_augmented))
# X_test = X_augmented[:num_test]
# y_test = y_augmented[:num_test]
# X_train = X_augmented[num_test:]
# y_train = y_augmented[num_test:]
# Pad and convert lists to numpy arrays
X_original = pad_sequences(X_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
y_original = pad_sequences(y_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
X_augmented = pad_sequences(X_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
y_augmented = pad_sequences(y_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')

num_test = int(0.2 * len(X_original))
X_test = X_original[:num_test]
y_test = y_original[:num_test]
X_train = np.concatenate([X_original[num_test:], X_augmented])
y_train = np.concatenate([y_original[num_test:], y_augmented])

scaler_X.fit(X_train.reshape(-1, X_train.shape[-1]))
scaler_y.fit(y_train.reshape(-1, 1))

joblib.dump(scaler_X, 'scaler_X.pkl')
joblib.dump(scaler_y, 'scaler_y.pkl')

['scaler_y.pkl']

In [7]:
def process_file_df(file_path):
    df = pd.read_csv(file_path)

    df['Time'] = df['Time'] - df['Time'].iloc[0]

    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)
    df['Acceleration'] = df['Speed'].diff().fillna(0)

    # features = df[['Engine speed', 'Throttle position', 'Accelerator pedal position', 'Speed']]
    features = df[['Engine speed', 'Speed', 'slope', 'Acceleration']]
    target = df['Momentary fuel consumption']

    features = features.iloc[:SEQUENCE_LENGTH]
    target = target.iloc[:SEQUENCE_LENGTH]
    # min_value = df[['Engine speed', 'Speed', 'slope', 'Acceleration','Momentary fuel consumption']].min()
    # max_value = df[['Engine speed', 'Speed', 'slope', 'Acceleration','Momentary fuel consumption']].max()
    # print(min_value,'minn\n\n')
    # print(max_value , 'maxx\n\n')
    return features.values, target.values ,df

In [9]:
basefolderpath = '/content/'
outputfilepath = os.path.join(basefolderpath , 'min_max_val.txt')
with open(outputfilepath,'w') as file:
  for i in range(6):
    if i == 5 :
      folder_path = os.path.join(base_folder_path, f'data_aug(3_slices_with_repeated)_cluster_{i}')
      for filename in os.listdir(folder_path):
          if filename.endswith('.csv'):
              file_path = os.path.join(folder_path, filename)
              features, target,df = process_file_df(file_path)
              # df = pd.read_csv(file_path)
              min_value = df[['Engine speed', 'Speed', 'slope', 'Acceleration','Momentary fuel consumption']].min()
              max_value = df[['Engine speed', 'Speed', 'slope', 'Acceleration','Momentary fuel consumption']].max()
              file.write(f'file : {filename}\n')
              file.write('min of each col :\n')
              file.write(min_value.to_string() + '\n')
              file.write('max of each col :\n')
              file.write(max_value.to_string() + '\n')
              file.write('\n'+ '='*50 + '\n')
              # print(f'file : {filename}')
              # print('min of each col :')
              # print(min_value)
              # print('max of each col :')
              # print(max_value)
              # print('\n'+ '='*50 + '\n')


In [ ]:
scaler_X.data_max_


array([4.8560000e+03, 1.2500000e+02, 4.2990656e+00, 9.4000000e+01],
      dtype=float32)

In [ ]:
scaler_X.data_min_

array([  0.       ,   0.       ,  -5.8295965, -97.       ], dtype=float32)

In [ ]:
scaler_y.data_max_


array([11882.25], dtype=float32)

In [ ]:
scaler_y.data_min_

array([-1576.812], dtype=float32)

In [ ]:
print(X_augmented.shape)

(20985, 600, 4)


In [ ]:
X_original

array([[[ 2.3510000e+03,  7.4000000e+01, -1.0218978e+00,  0.0000000e+00],
        [ 2.3562500e+03,  7.4000000e+01, -1.0218978e+00,  0.0000000e+00],
        [ 2.3582500e+03,  7.4000000e+01, -1.0218978e+00,  0.0000000e+00],
        ...,
        [ 2.3252500e+03,  9.8000000e+01, -1.1786371e+00,  1.0000000e+00],
        [ 2.3482500e+03,  9.7000000e+01, -1.1786371e+00, -1.0000000e+00],
        [ 2.3037500e+03,  9.7000000e+01, -1.1786371e+00,  0.0000000e+00]],

       [[ 3.4060000e+03,  1.0800000e+02, -4.2492917e-01,  0.0000000e+00],
        [ 3.4085000e+03,  1.0800000e+02, -4.2492917e-01,  0.0000000e+00],
        [ 3.4205000e+03,  1.0900000e+02, -4.2492917e-01,  1.0000000e+00],
        ...,
        [ 1.7527500e+03,  7.4000000e+01,  6.1387354e-01,  2.0000000e+00],
        [ 1.7810000e+03,  7.4000000e+01,  6.1387354e-01,  0.0000000e+00],
        [ 1.7852500e+03,  7.5000000e+01,  6.1387354e-01,  1.0000000e+00]],

       [[ 2.6627500e+03,  8.4000000e+01, -2.9965752e-01,  0.0000000e+00],
        

In [ ]:
X_augmented

array([[[ 2.3510000e+03,  7.4000000e+01, -1.0218978e+00,  0.0000000e+00],
        [ 2.3562500e+03,  7.4000000e+01, -1.0218978e+00,  0.0000000e+00],
        [ 2.3582500e+03,  7.4000000e+01, -1.0218978e+00,  0.0000000e+00],
        ...,
        [ 2.0722500e+03,  8.7000000e+01, -6.1302680e-01,  0.0000000e+00],
        [ 2.0765000e+03,  8.7000000e+01, -1.1433238e+00,  0.0000000e+00],
        [ 2.0995000e+03,  8.8000000e+01, -1.1433238e+00,  1.0000000e+00]],

       [[ 1.8650000e+03,  4.1000000e+01, -1.2152778e+00,  0.0000000e+00],
        [ 1.8917500e+03,  4.2000000e+01, -1.2152778e+00,  1.0000000e+00],
        [ 1.8460000e+03,  4.1000000e+01, -1.2152778e+00, -1.0000000e+00],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00]],

       [[ 8.7700000e+02,  0.0000000e+00, -2.4154589e+00,  0.0000000e+00],
        

In [10]:
def pad_and_normalize(data, scaler, sequence_length=SEQUENCE_LENGTH):
    padded_data = pad_sequences(data, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')
    # normalized_data = scaler.transform(padded_data.reshape(-1, padded_data.shape[-1])).reshape(padded_data.shape)
    return padded_data

In [11]:
X_train_normalized = pad_and_normalize(X_train, scaler_X)
y_train_normalized = y_train
X_test_normalized = pad_and_normalize(X_test, scaler_X)
y_test_normalized = y_test

print('X_train_normalized shape:', X_train_normalized.shape)
print('y_train_normalized shape:', y_train_normalized.shape)
print('X_test_normalized shape:', X_test_normalized.shape)
print('y_test_normalized shape:', y_test_normalized.shape)


X_train_normalized shape: (21009, 600, 4)
y_train_normalized shape: (21009, 600)
X_test_normalized shape: (5, 600, 4)
y_test_normalized shape: (5, 600)


#Manual normalized

In [12]:
min_val_x = [0,0,-10,-150]
max_val_x = [8000,150,10,150]

min_val_y = [-2000]
max_val_y = [20000]

for i in range(X_train_normalized.shape[-1]):
  X_train_normalized[:,:,i] = (X_train_normalized[:,:,i] - min_val_x[i])/(max_val_x[i]-min_val_x[i])

for i in range(X_test_normalized.shape[-1]):
  X_test_normalized[:,:,i] = (X_test_normalized[:,:,i] - min_val_x[i])/(max_val_x[i]-min_val_x[i])

y_train_normalized = (y_train-(-2000))/(20000 - (-2000))
y_test_normalized = (y_test-(-2000))/(20000 - (-2000))


In [13]:
X_train_normalized

array([[[0.34809375, 0.41333333, 0.48086125, 0.5       ],
        [0.3811875 , 0.44666666, 0.48086125, 0.51666665],
        [0.40821874, 0.48      , 0.48086125, 0.51666665],
        ...,
        [0.2606875 , 0.43333334, 0.53072196, 0.5       ],
        [0.25884375, 0.44      , 0.53072196, 0.50333333],
        [0.25721875, 0.43333334, 0.53072196, 0.49666667]],

       [[0.1874375 , 0.        , 0.703125  , 0.5       ],
        [0.3055625 , 0.01333333, 0.703125  , 0.50666666],
        [0.32496876, 0.02      , 0.703125  , 0.50333333],
        ...,
        [0.33846876, 0.4       , 0.4920509 , 0.5       ],
        [0.334625  , 0.39333335, 0.4920509 , 0.49666667],
        [0.334875  , 0.39333335, 0.4920509 , 0.5       ]],

       [[0.32746875, 0.55333334, 0.5228484 , 0.5       ],
        [0.32678124, 0.55333334, 0.5228484 , 0.5       ],
        [0.33071876, 0.56      , 0.5228484 , 0.50333333],
        ...,
        [0.09403125, 0.        , 0.5485437 , 0.5       ],
        [0.09465625, 0.      

In [19]:
import keras
from tensorflow.keras.regularizers import l2
EPOCHS = 200
LEARNING_RATE = 1e-5

model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True, input_shape=(SEQUENCE_LENGTH, X_train_normalized.shape[-1]),kernel_regularizer=l2(0.01),recurrent_regularizer=l2(0.01))),
    Dropout(0.2),
    Bidirectional(LSTM(64, return_sequences=True,kernel_regularizer=l2(0.01),recurrent_regularizer=l2(0.01))),
    Dropout(0.2),
    Dense(1)
])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error')

model.fit(X_train_normalized, y_train_normalized, epochs=EPOCHS, batch_size=64, validation_split=0.2,callbacks=[callback])


Epoch 1/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 29s 97ms/step - loss: 6.0522 - val_loss: 5.7579
Epoch 2/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 24s 93ms/step - loss: 5.6714 - val_loss: 5.4090
Epoch 3/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 25s 93ms/step - loss: 5.3269 - val_loss: 5.0786
Epoch 4/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 41s 94ms/step - loss: 5.0007 - val_loss: 4.7656
Epoch 5/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 41s 96ms/step - loss: 4.6917 - val_loss: 4.4691
Epoch 6/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 40s 93ms/step - loss: 4.3990 - val_loss: 4.1882
Epoch 7/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step - loss: 4.1216 - val_loss: 3.9220
Epoch 8/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 41s 91ms/step - loss: 3.8588 - val_loss: 3.6698
Epoch 9/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 25s 95ms/step - loss: 3.6098 - val_loss: 3.4307
Epoch 10/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 24s 92ms/step - loss: 3.3738 - val_loss: 3.2042
Epoch 11/200
263/263 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step - loss: 3.1502 - val_loss: 2.9896
Epoch 12/200
263/26

In [20]:
model.save('modelBLSTM_new_majid_added.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save('/content/drive/My Drive/model.h5')

In [21]:
test_loss = model.evaluate(X_test_normalized, y_test_normalized)
print(f"Test Loss: {test_loss}")

y_pred_normalized = model.predict(X_test_normalized)

y_test_inv = scaler_y.inverse_transform(y_test_normalized.reshape(-1, SEQUENCE_LENGTH))
y_pred_inv = scaler_y.inverse_transform(y_pred_normalized.reshape(-1, SEQUENCE_LENGTH))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.0030
Test Loss: 0.002978096017614007
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step


In [22]:
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 509227.09375


In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt

if not os.path.exists(PLOT_SAVE_DIR):
    os.makedirs(PLOT_SAVE_DIR)

for i in range(len(X_test)):
    # Convert momentary to cumulative
    cumulative_actuals = np.cumsum(y_test_inv[i], axis=0)  # assuming y_test_inv is already the inverse transformed data
    cumulative_predictions = np.cumsum(y_pred_inv[i], axis=0)  # assuming y_pred_inv is already the inverse transformed predictions

    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_actuals, label='Actual Trip Fuel Consumption')
    plt.plot(cumulative_predictions, label='Predicted Trip Fuel Consumption')
    plt.title(f'Trip {i + 1}: Actual vs Predicted Fuel Consumption')
    plt.xlabel('Time Steps')
    plt.ylabel('Trip Fuel Consumption (uL)')
    plt.legend()

    plot_filename = os.path.join(PLOT_SAVE_DIR, f'trip_{i + 1}_actual_vs_predicted.png')
    plt.savefig(plot_filename)
    plt.close()  # Close the plot to save memory



In [24]:
!zip -r data.zip predicted_vs_actual_plots

  adding: predicted_vs_actual_plots/ (stored 0%)
  adding: predicted_vs_actual_plots/trip_1_actual_vs_predicted.png (deflated 9%)
  adding: predicted_vs_actual_plots/trip_2_actual_vs_predicted.png (deflated 10%)
  adding: predicted_vs_actual_plots/trip_4_actual_vs_predicted.png (deflated 9%)
  adding: predicted_vs_actual_plots/trip_5_actual_vs_predicted.png (deflated 9%)
  adding: predicted_vs_actual_plots/trip_3_actual_vs_predicted.png (deflated 9%)
